In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib import colormaps
from scipy import stats
from matplotlib import rcParams
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import geopandas as gpd
from mpl_toolkits.axes_grid1 import make_axes_locatable
from geopandas.tools import overlay
import seaborn as sns
from scipy.stats import f_oneway
from scipy.stats import linregress
from sklearn.linear_model import LinearRegression

In [ ]:
df_site = pd.read_csv('itrdb_dem_landconr_values.csv')
df_site['year_difference']=df_site['end_year']-df_site['strat_year']
df_siteEPS = df_site[(df_site['eps'] >= 0.85) & (df_site['snr'] > 3) & 
            (df_site['LandCver'] == 10) & (df_site['end_year'] >= 1901) &
                (df_site['year_difference'] >= 30) & (df_site['end_year'] <= 2022)]
species_counts = df_siteEPS['Tree_Speci'].value_counts()
selected_species = species_counts[species_counts >= 20].index.tolist()
len(selected_species)
df_siteEPS_M_S = df_siteEPS[df_siteEPS['Tree_Speci'].isin(selected_species)]
df_siteEPS_M_S1 = df_siteEPS_M_S.loc[:,('site','DEM','Latitude','Longitude','Tree_Speci','end_year','strat_year','ai_v3')]
conditions = [
    df_siteEPS_M_S1['ai_v3'] < 0.65,
    df_siteEPS_M_S1['ai_v3'] >= 0.65
]
choices = ['Arid', 'Humid']
df_siteEPS_M_S1['AI_Class'] = np.select(conditions, choices)
df_siteEPS_M_S1.to_csv('sample_site.csv',index=False)
print(df_siteEPS_M_S1['AI_Class'].value_counts())

plt.style.use('default')
plt.rcParams["xtick.labelsize"] = 12
plt.rcParams["ytick.labelsize"] = 12
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 12
plt.figure(figsize=(2,3.5),dpi=100)
ai_class_counts = df_siteEPS_M_S1['AI_Class'].value_counts()
ai_class_counts.plot(kind='bar', color='skyblue')
plt.xlabel('Aridity Class',fontsize=12)
plt.ylabel('Count',fontsize=12)
plt.xticks(rotation=0)
plt.savefig('Fig1C.jpg',dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
df_species = pd.read_excel('40tree.xlsx')
df_species_O = df_species.sort_values('order')
species_dict = dict(zip(df_species['Tree_Speci'], df_species['Clade']))
df_siteEPS_M_S1['Species_group'] = df_siteEPS_M_S1['Tree_Speci'].map(species_dict)
df_siteEPS_M_S1['Species_group'] = df_siteEPS_M_S1['Species_group'].fillna('Unknown')

df = pd.DataFrame(index=range(1901, 2016))

plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 18
plt.rcParams["xtick.labelsize"] = 16
plt.rcParams["ytick.labelsize"] = 16

for species in df_siteEPS_M_S1['Tree_Speci'].unique():
    df_species = df_siteEPS_M_S1[df_siteEPS_M_S1['Tree_Speci'] == species]

    for year in range(1901, 2016):
        df.loc[year, species] = len(df_species[df_species['end_year'] >= year])

df.fillna(0, inplace=True)

species_groups = df_siteEPS_M_S1[['Tree_Speci', 'Species_group']].drop_duplicates()
species_groups = species_groups.set_index('Tree_Speci')

new_order = df_species_O[df_species_O['Tree_Speci'].isin(df.columns)]['Tree_Speci']
df = df.reindex(columns= new_order)

n_angiosperm = len(species_groups[species_groups['Species_group'] == 'Angiosperms'])
n_gymnosperm = len(species_groups[species_groups['Species_group'] == 'Gymnosperms'])

colors = plt.cm.RdBu(np.linspace(0, 0.5, n_gymnosperm).tolist() + np.linspace(0.5, 1, n_angiosperm).tolist())
)
cmap = ListedColormap(colors)

plt.figure(figsize=(10, 6))
ax = df.plot(kind='bar', width=0.8, stacked=True, colormap=cmap, ax=plt.gca())

plt.xlabel('Year', fontsize=18)
plt.ylabel('Number of site', fontsize=18)

xticks = [1901, 1920, 1940, 1960, 1980, 2000, 2015]
ax.set_xticks([df.index.get_loc(i) for i in xticks])
ax.set_xticklabels(xticks)

ax.legend().remove()

norm = plt.Normalize(vmin=0, vmax=len(df.columns)-1)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([]) 
cbar = plt.colorbar(sm, ax=ax, pad=0.02,)
cbar.set_ticks(np.arange(len(df.columns)))
cbar.set_ticklabels(df.columns,fontsize=12)
plt.tight_layout()
plt.savefig('Fig2b.jpg', dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
gdf = gpd.GeoDataFrame(df_siteEPS_M_S1, geometry=gpd.points_from_xy(df_siteEPS_M_S1.Longitude, df_siteEPS_M_S1.Latitude))
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
fig, ax = plt.subplots(figsize=(8, 4))

plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 12
plt.rcParams["xtick.labelsize"] = 12
plt.rcParams["ytick.labelsize"] = 12
ax.set_ylim(-20, 90)
world.boundary.plot(ax=ax, linewidth=0.5, color='black')
gdf['Tree_Speci'] = gdf['Tree_Speci'].astype('category')
species_groups = df_siteEPS_M_S1[['Tree_Speci', 'Species_group']].drop_duplicates()
gdf['Species_group_rank'] = gdf['Tree_Speci'].map(species_groups.set_index('Tree_Speci')['Species_group'].replace({'Angiosperms': 0, 'Gymnosperms': 1}))
gdf = gdf.sort_values(by='Species_group_rank')
n_angiosperm = len(species_groups[species_groups['Species_group'] == 'Angiosperms'])
n_gymnosperm = len(species_groups[species_groups['Species_group'] == 'Gymnosperms'])
colors = plt.cm.RdBu(np.linspace(0, 0.5, n_gymnosperm).tolist() + np.linspace(0.5, 1, n_angiosperm).tolist())
cmap = ListedColormap(colors)
gdf.plot(column='Tree_Speci', ax=ax, legend=False, cmap=cmap, markersize=4)
ax.grid(True, linestyle='--', color='black',linewidth=0.4)
divider = make_axes_locatable(ax)
cax = divider.append_axes("bottom", size="8%", pad=0.3)

sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=0, vmax=len(gdf['Tree_Speci'].cat.categories)-1))
cbar = fig.colorbar(sm, cax=cax, orientation='horizontal')
sorted_categories = new_order
cbar.set_ticks(np.arange(len(sorted_categories)))
cbar.set_ticklabels(sorted_categories)
cbar.ax.set_xticklabels(cbar.ax.get_xticklabels(), rotation=90, ha='right')

plt.tight_layout()
plt.savefig('Distribution tree species.jpg',dpi=600,bbox_inches='tight')
plt.show()

In [ ]:
Tmax = pd.read_csv('annual_Tmax.csv').dropna()
list_site = pd.merge(df_siteEPS_M_S1,Tmax,on='site')['site']
list_site = pd.merge(df_siteEPS_M_S1,Tmax,on='site')['site']
from scipy.stats import linregress
from tqdm import tqdm
from scipy import stats
from sklearn.linear_model import LinearRegression

list_sites = []
list_β_Tmax = []
for i in tqdm(range(len(list_site)), desc="Calculating Pearson Correlation"):
    site_i = list_site[i]
    std_path = 'E:\experiment\DTR_AI_growth\data\ITRDB\FZS_STD_crn\\'+ site_i + '.rwlcrn.csv'
    df_rwl_i = pd.read_csv(std_path, index_col=0)
    year_i = df_rwl_i.index.to_list()
    list_range = [i for i in range(1901, 2016)]
    intersection = list(set(year_i) & set(list_range))
    if len(intersection)>= 10:
        std_i = df_rwl_i.loc[intersection,'std'].to_list()
        Tmax_i = Tmax.set_index('site').loc[site_i,[f'{year}' for year in intersection]]
        Tmax_i = np.array(Tmax_i).reshape(-1, 1)
        model = LinearRegression()
        model.fit(Tmax_i, std_i)
        list_β_Tmax.append(model.coef_)
        list_sites.append(site_i)
df_tmaxβ = pd.DataFrame(list_β_Tmax)
df_tmaxβ.columns = ['β']
df_tmaxβ.insert(0,'site',list_sites)
df_tmaxβcs = pd.merge(df_siteEPS_M_S1,df_tmaxβ,on='site')
def calculate_confidence_interval(df, column):
    lower_bound = np.percentile(df[column], 2.5)
    upper_bound = np.percentile(df[column], 97.5)
    df_95 = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    mean_value = np.mean(df_95[column])
    print("95% Confidence Interval ({}): [{:.4f}, {:.4f}]".format(column, lower_bound, upper_bound))
    print("Mean ({}): {:.4f}".format(column, mean_value))
    return df_95
grouped = df_tmaxβcs.groupby('AI_Class')
anova_results_dr = {}
anova_results_wr = {}
group_humid = grouped.get_group('Humid')
group_arid = grouped.get_group('Arid')
group_humid95 = calculate_confidence_interval(group_humid, 'β')
group_arid95 = calculate_confidence_interval(group_arid, 'β')

group_95_Tmax = pd.concat([group_humid95,group_arid95],axis=0)
group_95_Tmax['Climate factors'] = 'Tmax'
f_statistic_dr, p_value_dr = f_oneway(group_humid95['β'], group_arid95['β'])
anova_results_dr['β'] = {'F-statistic': f_statistic_dr, 'p-value': p_value_dr}
for key, value in anova_results_dr.items():
    print(f"{key}: F-statistic: {value['F-statistic']}, p-value: {value['p-value']}")
    plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 16
plt.rcParams["xtick.labelsize"] = 16
plt.rcParams["ytick.labelsize"] = 16
plt.figure(figsize=(4,3.5),dpi=100)
palette=[(0/255, 92/255, 230/255),(255/255, 0/255, 0/255)]
alpha = 0.5 
sns.boxplot(x="AI_Class", y="β", data=group_95_Tmax,palette=palette
            ,  fill=False, gap=.2, boxprops=dict(alpha=alpha)
            , whiskerprops=dict(alpha=alpha)
            , capprops=dict(alpha=alpha), medianprops=dict(alpha=alpha))
plt.axhline(0, color='black', linestyle='--',linewidth=.6)
plt.yticks(np.arange(-0.4, 0.5, 0.2)) 
plt.legend().set_visible(False)
plt.xlabel('')
plt.ylabel('β Tmax-Growth',fontsize=18)
plt.text(0.5, 0.37, '***', ha='center', va='center')
plt.text('Humid', 0.33,  f'n= {len(group_humid95)}', ha='center', va='center')
plt.text('Arid', 0.33,  f'n= {len(group_arid95)}', ha='center', va='center')
plt.savefig('FigS7a.jpg',dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
Tmin = pd.read_csv('annual_Tmin.csv').dropna()
list_site = pd.merge(df_siteEPS_M_S1,Tmin,on='site')['site']
from scipy.stats import linregress
from tqdm import tqdm
from scipy import stats
from sklearn.linear_model import LinearRegression

list_sites = []
list_β_Tmin = []
for i in tqdm(range(len(list_site)), desc="Calculating Pearson Correlation"):
    site_i = list_site[i]
    std_path = 'E:\experiment\DTR_AI_growth\data\ITRDB\FZS_STD_crn\\'+ site_i + '.rwlcrn.csv'
    df_rwl_i = pd.read_csv(std_path, index_col=0)
    year_i = df_rwl_i.index.to_list()
    list_range = [i for i in range(1901, 2016)]
    intersection = list(set(year_i) & set(list_range))
    if len(intersection)>= 10:
        std_i = df_rwl_i.loc[intersection,'std'].to_list()
        Tmin_i = Tmin.set_index('site').loc[site_i,[f'{year}' for year in intersection]]
        Tmin_i = np.array(Tmin_i).reshape(-1, 1)
        model = LinearRegression()
        model.fit(Tmin_i, std_i)
        list_β_Tmin.append(model.coef_)
        list_sites.append(site_i)
df_β_Tmin = pd.DataFrame(list_β_Tmin)
df_β_Tmin.columns = ['β']
df_β_Tmin.insert(0,'site',list_sites)
df_β_Tmincs = pd.merge(df_siteEPS_M_S1,df_β_Tmin,on='site')
grouped = df_β_Tmincs.groupby('AI_Class')
anova_results_dr = {}
anova_results_wr = {}
group_humid = grouped.get_group('Humid')
group_arid = grouped.get_group('Arid')
group_humid95 = calculate_confidence_interval(group_humid, 'β')
group_arid95 = calculate_confidence_interval(group_arid, 'β')

group_95_tmin = pd.concat([group_humid95,group_arid95],axis=0)
group_95_tmin['Climate factors'] = 'Tmin'

f_statistic_dr, p_value_dr = f_oneway(group_humid['β'], group_arid['β'])
anova_results_dr['β'] = {'F-statistic': f_statistic_dr, 'p-value': p_value_dr}

for key, value in anova_results_dr.items():
    print(f"{key}: F-statistic: {value['F-statistic']}, p-value: {value['p-value']}")
    plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 16
plt.rcParams["xtick.labelsize"] = 16
plt.rcParams["ytick.labelsize"] = 16
plt.figure(figsize=(4,3.5),dpi=100)
palette=[(0/255, 92/255, 230/255),(255/255, 0/255, 0/255)]
alpha = 0.5 
sns.boxplot(x="AI_Class", y="β", data=group_95_tmin,palette=palette
            ,  fill=False, gap=.2, boxprops=dict(alpha=alpha)
            , whiskerprops=dict(alpha=alpha)
            , capprops=dict(alpha=alpha), medianprops=dict(alpha=alpha))
plt.axhline(0, color='black', linestyle='--',linewidth=.6)
plt.yticks(np.arange(-0.4, 0.5, 0.2)) 
plt.legend().set_visible(False)
plt.xlabel('')
plt.ylabel('β Tmin-Growth',fontsize=18)
plt.text(0.5, 0.37, '***', ha='center', va='center')
plt.text('Humid', 0.33,  f'n= {len(group_humid95)}', ha='center', va='center')
plt.text('Arid', 0.33,  f'n= {len(group_arid95)}', ha='center', va='center')
plt.savefig('FigS7e.jpg',dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
Pre = pd.read_csv('annual_pre.csv').dropna()
list_site = pd.merge(df_siteEPS_M_S1,Pre,on='site')['site']
list_sites = []
list_β_Pre = []
for i in tqdm(range(len(list_site)), desc="Calculating Pearson Correlation"):
    site_i = list_site[i]
    std_path = 'E:\experiment\DTR_AI_growth\data\ITRDB\FZS_STD_crn\\'+ site_i + '.rwlcrn.csv'
    df_rwl_i = pd.read_csv(std_path, index_col=0)
    year_i = df_rwl_i.index.to_list()
    list_range = [i for i in range(1901, 2016)]
    intersection = list(set(year_i) & set(list_range))
    if len(intersection)>= 10:
        std_i = df_rwl_i.loc[intersection,'std'].to_list()
        Pre_i = Pre.set_index('site').loc[site_i,[f'{year}' for year in intersection]]
        Pre_i = np.array(Pre_i).reshape(-1, 1)
        model = LinearRegression()
        model.fit(Pre_i, std_i)
        list_β_Pre.append(model.coef_)
        list_sites.append(site_i)

df_β_Pre = pd.DataFrame(list_β_Pre)
df_β_Pre.columns = ['β']
df_β_Pre.insert(0,'site',list_sites)
df_β_Precs = pd.merge(df_siteEPS_M_S1,df_β_Pre,on='site')
import seaborn as sns
from scipy.stats import f_oneway
import matplotlib.pyplot as plt
grouped = df_β_Precs.groupby('AI_Class')
anova_results_dr = {}
anova_results_wr = {}
group_humid = grouped.get_group('Humid')
group_arid = grouped.get_group('Arid')
group_humid95 = calculate_confidence_interval(group_humid, 'β')
group_arid95 = calculate_confidence_interval(group_arid, 'β')
group_95_pre = pd.concat([group_humid95,group_arid95],axis=0)
group_95_pre['Climate factors'] = 'Pre'
f_statistic_dr, p_value_dr = f_oneway(group_humid95['β'], group_arid95['β'])
anova_results_dr['β'] = {'F-statistic': f_statistic_dr, 'p-value': p_value_dr}
for key, value in anova_results_dr.items():
    print(f"{key}: F-statistic: {value['F-statistic']}, p-value: {value['p-value']}")
    plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 16
plt.rcParams["xtick.labelsize"] = 16
plt.rcParams["ytick.labelsize"] = 16
plt.figure(figsize=(4,3.5),dpi=100)
palette=[(0/255, 92/255, 230/255),(255/255, 0/255, 0/255)]
alpha = 0.5 
sns.boxplot(x="AI_Class", y="β", data=group_95_pre,palette=palette
            ,  fill=False, gap=.2, boxprops=dict(alpha=alpha)
            , whiskerprops=dict(alpha=alpha)
            , capprops=dict(alpha=alpha), medianprops=dict(alpha=alpha))
plt.axhline(0, color='black', linestyle='--',linewidth=.6)
plt.yticks(np.arange(-0.002, 0.005, 0.002)) 
plt.legend().set_visible(False)
plt.xlabel('')
plt.ylabel('β Pre-Growth',fontsize=18)
plt.text(0.5, 0.0037, '***', ha='center', va='center')
plt.text('Humid', 0.0033,  f'n= {len(group_humid95)}', ha='center', va='center')
plt.text('Arid', 0.0033,  f'n= {len(group_arid95)}', ha='center', va='center')
plt.savefig('Fig.S7F.jpg',dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
Temp = pd.read_csv('annual_temp.csv').dropna()
list_site = pd.merge(df_siteEPS_M_S1,Temp,on='site')['site']
list_sites = []
list_β_Temp = []
for i in tqdm(range(len(list_site)), desc="Calculating Pearson Correlation"):
    site_i = list_site[i]
    std_path = 'E:\experiment\DTR_AI_growth\data\ITRDB\FZS_STD_crn\\'+ site_i + '.rwlcrn.csv'
    df_rwl_i = pd.read_csv(std_path, index_col=0)
    year_i = df_rwl_i.index.to_list()
    list_range = [i for i in range(1901, 2016)]
    intersection = list(set(year_i) & set(list_range))
    if len(intersection)>= 10:
        std_i = df_rwl_i.loc[intersection,'std'].to_list()
        Temp_i = Temp.set_index('site').loc[site_i,[f'{year}' for year in intersection]]
        Temp_i = np.array(Temp_i).reshape(-1, 1)
        model = LinearRegression()
        model.fit(Temp_i, std_i)
        list_β_Temp.append(model.coef_)
        list_sites.append(site_i)

df_β_Temp = pd.DataFrame(list_β_Temp)
df_β_Temp.columns = ['β']
df_β_Temp.insert(0,'site',list_sites)
df_β_Tempcs = pd.merge(df_siteEPS_M_S1,df_β_Temp,on='site')
grouped = df_β_Tempcs.groupby('AI_Class')
anova_results_dr = {}
anova_results_wr = {}
group_humid = grouped.get_group('Humid')
group_arid = grouped.get_group('Arid')
group_humid95 = calculate_confidence_interval(group_humid, 'β')
group_arid95 = calculate_confidence_interval(group_arid, 'β')
group_95_temp = pd.concat([group_humid95,group_arid95],axis=0)
group_95_temp['Climate factors'] = 'temp'
f_statistic_dr, p_value_dr = f_oneway(group_humid['β'], group_arid['β'])
anova_results_dr['β'] = {'F-statistic': f_statistic_dr, 'p-value': p_value_dr}
for key, value in anova_results_dr.items():
    print(f"{key}: F-statistic: {value['F-statistic']}, p-value: {value['p-value']}")
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 16
plt.rcParams["xtick.labelsize"] = 16
plt.rcParams["ytick.labelsize"] = 16
plt.figure(figsize=(4,3.5),dpi=100)
palette=[(0/255, 92/255, 230/255),(255/255, 0/255, 0/255)]
alpha = 0.5 
sns.boxplot(x="AI_Class", y="β", data=group_95_temp,palette=palette
            ,  fill=False, gap=.2, boxprops=dict(alpha=alpha)
            , whiskerprops=dict(alpha=alpha)
            , capprops=dict(alpha=alpha), medianprops=dict(alpha=alpha))
plt.axhline(0, color='black', linestyle='--',linewidth=.6)
plt.yticks(np.arange(-0.4, 0.5, 0.2)) 
plt.legend().set_visible(False)
plt.xlabel('')
plt.ylabel('β Temp-Growth',fontsize=18)
plt.text(0.5, 0.37, '***', ha='center', va='center')
plt.text('Humid', 0.33,  f'n= {len(group_humid95)}', ha='center', va='center')
plt.text('Arid', 0.33,  f'n= {len(group_arid95)}', ha='center', va='center')
plt.savefig('FigS7C.jpg',dpi=300, bbox_inches='tight')
plt.show()    

In [ ]:
dtr = pd.read_csv('annual_dtr.csv').dropna()
list_site = pd.merge(df_siteEPS_M_S1,dtr,on='site')['site']
from scipy.stats import linregress
from tqdm import tqdm
from scipy import stats
from sklearn.linear_model import LinearRegression
import scipy.stats

list_sites = []
list_β_dtr = []
list_R = []
for i in tqdm(range(len(list_site)), desc="Calculating Pearson Correlation"):
    site_i = list_site[i]
    std_path = 'E:\experiment\DTR_AI_growth\data\ITRDB\FZS_STD_crn\\'+ site_i + '.rwlcrn.csv'
    df_rwl_i = pd.read_csv(std_path, index_col=0)
    year_i = df_rwl_i.index.to_list()
    list_range = [i for i in range(1901, 2016)]
    intersection = list(set(year_i) & set(list_range))
    if len(intersection)>= 10:
        std_i = df_rwl_i.loc[intersection,'std'].to_list()
        dtr_i = dtr.set_index('site').loc[site_i,[f'{year}' for year in intersection]]
        dtr_i1=dtr.set_index('site').loc[site_i,[f'{year}' for year in intersection]]
        dtr_i = np.array(dtr_i).reshape(-1, 1)
        model = LinearRegression()
        model.fit(dtr_i, std_i)
        list_β_dtr.append(model.coef_)
        slope, intercept, r_value, p_value, std_err= scipy.stats.linregress(dtr_i1.to_list(), std_i)
        list_R.append(r_value)
        list_sites.append(site_i)

df_β_dtr = pd.DataFrame(list_β_dtr)
df_β_dtr.columns = ['β']
df_β_dtr.insert(0,'site',list_sites)
df_β_dtr.insert(1,'r_squared',list_R)
df_β_dtrcs = pd.merge(df_siteEPS_M_S1,df_β_dtr,on='site')
grouped = df_β_dtrcs.groupby('AI_Class')
anova_results_dr = {}
anova_results_wr = {}
group_humid = grouped.get_group('Humid')
group_arid = grouped.get_group('Arid')
group_humid95 = calculate_confidence_interval(group_humid, 'β')
group_arid95 = calculate_confidence_interval(group_arid, 'β')
group_95_dtr = pd.concat([group_humid95,group_arid95],axis=0)
group_95_dtr['Climate factors'] = 'dtr'
ue
f_statistic_dr, p_value_dr = f_oneway(group_humid['β'], group_arid['β'])
anova_results_dr['β'] = {'F-statistic': f_statistic_dr, 'p-value': p_value_dr}

for key, value in anova_results_dr.items():
    print(f"{key}: F-statistic: {value['F-statistic']}, p-value: {value['p-value']}")
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 16
plt.rcParams["xtick.labelsize"] = 16
plt.rcParams["ytick.labelsize"] = 16
plt.figure(figsize=(4,3.5),dpi=100)
palette=[(0/255, 92/255, 230/255),(255/255, 0/255, 0/255)]
alpha = 0.5 
sns.boxplot(x="AI_Class", y="β", data=group_95_dtr,palette=palette
            ,  fill=False, gap=.2, boxprops=dict(alpha=alpha)
            , whiskerprops=dict(alpha=alpha)
            , capprops=dict(alpha=alpha), medianprops=dict(alpha=alpha))
plt.axhline(0, color='black', linestyle='--',linewidth=.6)
plt.yticks(np.arange(-0.4, 0.5, 0.2)) 
plt.legend().set_visible(False)
plt.xlabel('')
plt.ylabel('β DTR-Growth',fontsize=18)
plt.text(0.5, 0.37, '***', ha='center', va='center')
plt.text('Humid', 0.33,  f'n= {len(group_humid95)}', ha='center', va='center')
plt.text('Arid', 0.33,  f'n= {len(group_arid95)}', ha='center', va='center')
plt.savefig('FigS7b.jpg',dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
scPDSI = pd.read_csv('annual_scPDSI.csv').dropna()
list_site = pd.merge(df_siteEPS_M_S1,scPDSI,on='site')['site']
list_sites = []
list_β_scPDSI = []
for i in tqdm(range(len(list_site)), desc="Calculating Pearson Correlation"):
    site_i = list_site[i]
    std_path = 'E:\experiment\DTR_AI_growth\data\ITRDB\FZS_STD_crn\\'+ site_i + '.rwlcrn.csv'
    df_rwl_i = pd.read_csv(std_path, index_col=0)
    year_i = df_rwl_i.index.to_list()
    list_range = [i for i in range(1901, 2016)]
    intersection = list(set(year_i) & set(list_range))
    if len(intersection)>= 10:
        std_i = df_rwl_i.loc[intersection,'std'].to_list()
        scPDSI_i = scPDSI.set_index('site').loc[site_i,[f'{year}' for year in intersection]]
        scPDSI_i = np.array(scPDSI_i).reshape(-1, 1)
        model = LinearRegression()
        model.fit(scPDSI_i, std_i)
        list_β_scPDSI.append(model.coef_)
        list_sites.append(site_i)

df_β_scPDSI = pd.DataFrame(list_β_scPDSI)
df_β_scPDSI.columns = ['β']
df_β_scPDSI.insert(0,'site',list_sites)
df_β_scPDSIcs = pd.merge(df_siteEPS_M_S1,df_β_scPDSI,on='site')
grouped = df_β_scPDSIcs.groupby('AI_Class')
anova_results_dr = {}
anova_results_wr = {}
group_humid = grouped.get_group('Humid')
group_arid = grouped.get_group('Arid')
group_humid95 = calculate_confidence_interval(group_humid, 'β')
group_arid95 = calculate_confidence_interval(group_arid, 'β')

group_95_scPDSI = pd.concat([group_humid95,group_arid95],axis=0)
group_95_scPDSI['Climate factors'] = 'scPDSI'
f_statistic_dr, p_value_dr = f_oneway(group_humid['β'], group_arid['β'])
anova_results_dr['β'] = {'F-statistic': f_statistic_dr, 'p-value': p_value_dr}
for key, value in anova_results_dr.items():
    print(f"{key}: F-statistic: {value['F-statistic']}, p-value: {value['p-value']}")
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 16
plt.rcParams["xtick.labelsize"] = 16
plt.rcParams["ytick.labelsize"] = 16
plt.figure(figsize=(4,3.5),dpi=100)
palette=[(0/255, 92/255, 230/255),(255/255, 0/255, 0/255)]
alpha = 0.5 
sns.boxplot(x="AI_Class", y="β", data=group_95_scPDSI,palette=palette
            ,  fill=False, gap=.2, boxprops=dict(alpha=alpha)
            , whiskerprops=dict(alpha=alpha)
            , capprops=dict(alpha=alpha), medianprops=dict(alpha=alpha))
plt.axhline(0, color='black', linestyle='--',linewidth=.6)
plt.yticks(np.arange(-0.2, 0.5, 0.2)) 
plt.legend().set_visible(False)
plt.xlabel('')
plt.ylabel('β scPDSI-growth',fontsize=18)
plt.text(0.5, 0.37, '***', ha='center', va='center')
plt.text('Humid', 0.33,  f'n= {len(group_humid95)}', ha='center', va='center')
plt.text('Arid', 0.33,  f'n= {len(group_arid95)}', ha='center', va='center')
plt.savefig('Fig.S7d.jpg',dpi=300, bbox_inches='tight')
plt.show()    